# 1-03、
### 通过观察前面代码的输出结果，我们发现,代码是对DStream中的RDD分别处理的，没有将他们聚合在一起处理；
### 通过调用updateStateByKey方法可以实现将所有已接收到的RDD进行聚合处理

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext('local[1]')        # 将这里参数设置为local[1]    因为后面设置有检查点目录，使用超过1个内核会引起资源争夺导致超时

In [ ]:
from pyspark.streaming import StreamingContext
ssc = StreamingContext(sc, 30)       # 根据自己电脑的配置情况和处理数据的复杂程度，适当调整这个参数的大小。上课时设置的是5， 这里修改为30

In [ ]:
# 将新到来的值（new_values）和之前的值(last_num)相加
def word_count(new_values, last_sum):     
    if last_sum is None:
        last_sum = 0
    return sum(new_values, last_sum)     

In [ ]:
inputDStream = ssc.queueStream([['a', 'b', 'c'], ['a', 'c'], ['e', 'e', 'c']])
pairDStream = inputDStream.map(lambda x : (x, 1)).reduceByKey(lambda x, y : x + y)

# 使用updateStateByKey方法前，必须先设置checkpoint目录，用于保存之前处理过的RDD
ssc.checkpoint('checkpoints')
updateStateDStream = pairDStream.updateStateByKey(word_count)

updateStateDStream.pprint()

ssc.start()

In [ ]:
# ssc.awaitTermination(30)
# ssc.stop()